In [6]:
import random
from shapely.geometry import Point
import geopandas as gpd

# 대한민국 영역 Shapefile 경로
shapefile_path = "./sig.shp"

# DBF 파일의 인코딩을 확인한 후 올바른 인코딩을 사용하세요 (예: 'euc-kr')
encoding = 'euc-kr'

# Shapefile 읽어오기
gdf = gpd.read_file(shapefile_path, encoding=encoding)

# 좌표계(CRS) 설정
if gdf.crs is None:
    # 만약 CRS가 설정되지 않은 경우, EPSG:5179 (Korean TM)으로 설정한 후 EPSG:4326으로 변환
    gdf.set_crs(epsg=5179, inplace=True)

# EPSG:4326으로 좌표계 변환 (WGS84, GPS에서 사용하는 표준 좌표계)
gdf = gdf.to_crs(epsg=4326)

# 시/도 맵핑
sigungu_map = {
    '11': "서울특별시",
    '26': "부산광역시",
    '27': "대구광역시",
    '28': "인천광역시",
    '29': "광주광역시",
    '30': "대전광역시",
    '31': "울산광역시",
    '41': "경기도",
    '43': "충청북도",
    '44': "충청남도",
    '45': "전라북도",
    '46': "전라남도",
    '47': "경상북도",
    '48': "경상남도",
    '50': "제주도",
    '51': "강원도"
}

# 시/도 코드 목록 출력
def print_region_codes():
    print("다음 지역의 코드를 제외할 수 있습니다:")
    for code, name in sigungu_map.items():
        print(f"{code}: {name}")

# 사용자에게 제외할 지역 코드 입력받기
def get_excluded_codes():
    excluded_codes_input = input("제외할 지역의 코드를 쉼표로 구분하여 입력하세요: ").split(',')
    excluded_codes_input = [code.strip() for code in excluded_codes_input]
    # 유효한 코드만 필터링
    return excluded_codes_input

def confirm_excluded_codes(excluded_codes):
    excluded_provinces = [sigungu_map.get(code, "") for code in excluded_codes]
    excluded_provinces = [name for name in excluded_provinces if name]  # 빈 값 제거
    
    print("제외한 지역이 다음과 같습니까?")
    for name in excluded_provinces:
        print(f"{name}", end=' ')
        print()
    
    confirmation = input("맞다: Y, 틀리다: N")
    return confirmation == 'Y'

def validate_excluded_codes(excluded_codes):
    valid_codes = set(sigungu_map.keys())
    invalid_codes = set(excluded_codes) - valid_codes
    if invalid_codes:
        print(f"잘못된 지역 코드가 포함되어 있습니다: {', '.join(invalid_codes)}")
        return False
    return True

def main():
    print_region_codes()
    excluded_codes = get_excluded_codes()

    while not validate_excluded_codes(excluded_codes) or not confirm_excluded_codes(excluded_codes):
        excluded_codes = get_excluded_codes()
    
    excluded_codes = [code for code in excluded_codes if code in sigungu_map]  # 다시 필터링

    # 모든 시/도 코드와 제외된 시/도 코드의 집합을 생성
    all_codes = set(sigungu_map.keys())
    remaining_codes = all_codes - set(excluded_codes)

    if not remaining_codes:
        print("여행 가능한 구역이 없습니다.")
        return

    # 대한민국 영역 내에 포함되는지 확인하고, 속한 시/군/구를 반환하는 함수
    def generate_random_point_within_korea():
        within_korea = False
        point = None
        attempts = 0
        max_attempts = 100

        while not within_korea and attempts < max_attempts:
            attempts += 1

            # 랜덤 위도 및 경도 생성 (대한민국의 범위 내에서)
            latitude = round(random.uniform(33.091, 38.614), 3)
            longitude = round(random.uniform(124.511, 131.874), 3)

            # Point 객체 생성
            point = Point(longitude, latitude)

            # 지리 데이터 프레임의 각 폴리곤(시/군/구) 내에 있는지 확인
            for index, row in gdf.iterrows():
                polygon = row['geometry']
                if point.within(polygon):  # Point가 Polygon 내부에 있는지 확인
                    sig_cd = row['SIG_CD']
                    region = row['SIG_KOR_NM']
                    
                    # sig_cd에 따라 시/도를 추가
                    sigungu = sigungu_map.get(sig_cd[:2], "")
                    if sigungu and sig_cd[:2] not in excluded_codes:
                        full_region = f"{sigungu} {region}" if sigungu else region
                        return point, sig_cd, full_region  # 점과 SIG_CD 반환

        return None, None, None  # 최대 시도 횟수를 넘긴 경우

    # 랜덤한 지역을 찾기 위해 반복
    attempts = 0
    point, sig_cd, full_region = None, "", ""

    while point is None:
        point, sig_cd, full_region = generate_random_point_within_korea()
        attempts += 1

    # 제외된 지역 확인
    excluded_provinces = [sigungu_map.get(code, "") for code in excluded_codes]
    excluded_provinces = [name for name in excluded_provinces if name]  # 빈 값 제거

    if point:
        print("당첨 지역:", full_region)
        print("제외된 지역:", ", ".join(excluded_provinces))
        # print("뽑기 횟수:", attempts)

if __name__ == "__main__":
    main()


다음 지역의 코드를 제외할 수 있습니다:
11: 서울특별시
26: 부산광역시
27: 대구광역시
28: 인천광역시
29: 광주광역시
30: 대전광역시
31: 울산광역시
41: 경기도
43: 충청북도
44: 충청남도
45: 전라북도
46: 전라남도
47: 경상북도
48: 경상남도
50: 제주도
51: 강원도
제외한 지역이 다음과 같습니까?
부산광역시 
대구광역시 
인천광역시 
광주광역시 
대전광역시 
울산광역시 
경기도 
충청북도 
충청남도 
전라북도 
전라남도 
경상북도 
경상남도 
제주도 
강원도 
당첨 지역: 서울특별시 영등포구
제외된 지역: 부산광역시, 대구광역시, 인천광역시, 광주광역시, 대전광역시, 울산광역시, 경기도, 충청북도, 충청남도, 전라북도, 전라남도, 경상북도, 경상남도, 제주도, 강원도
제외된 기록:
서울특별시 영등포구


In [18]:
import random
from shapely.geometry import Point
import geopandas as gpd

# 대한민국 영역 Shapefile 경로
shapefile_path = "./sig.shp"

# DBF 파일의 인코딩을 확인한 후 올바른 인코딩을 사용하세요 (예: 'euc-kr')
encoding = 'euc-kr'

# Shapefile 읽어오기
gdf = gpd.read_file(shapefile_path, encoding=encoding)

# 좌표계(CRS) 설정
if gdf.crs is None:
    # 만약 CRS가 설정되지 않은 경우, EPSG:5179 (Korean TM)으로 설정한 후 EPSG:4326으로 변환
    gdf.set_crs(epsg=5179, inplace=True)

# EPSG:4326으로 좌표계 변환 (WGS84, GPS에서 사용하는 표준 좌표계)
gdf = gdf.to_crs(epsg=4326)

# 시/도 맵핑
sigungu_map = {
    '11': "서울특별시",
    '26': "부산광역시",
    '27': "대구광역시",
    '28': "인천광역시",
    '29': "광주광역시",
    '30': "대전광역시",
    '31': "울산광역시",
    '41': "경기도",
    '43': "충청북도",
    '44': "충청남도",
    '45': "전라북도",
    '46': "전라남도",
    '47': "경상북도",
    '48': "경상남도",
    '50': "제주도",
    '51': "강원도"
}

# 시/도 코드 목록 출력
def print_region_codes():
    print("다음 지역의 코드를 제외할 수 있습니다:")
    for code, name in sigungu_map.items():
        print(f"{code}: {name}")

# 사용자에게 제외할 지역 코드 입력받기
def get_excluded_codes():
    excluded_codes_input = input("제외할 지역의 코드를 쉼표로 구분하여 입력하세요: ").split(',')
    excluded_codes_input = [code.strip() for code in excluded_codes_input]
    # 유효한 코드만 필터링
    return excluded_codes_input

def confirm_excluded_codes(excluded_codes):
    excluded_provinces = [sigungu_map.get(code, "") for code in excluded_codes]
    excluded_provinces = [name for name in excluded_provinces if name]  # 빈 값 제거
    
    print("제외한 지역이 다음과 같습니까?")
    for name in excluded_provinces:
        print(f"{name}", end=' ')
        print()
    
    confirmation = input("맞다: Y, 틀리다: N")
    return confirmation == 'Y'

def validate_excluded_codes(excluded_codes):
    valid_codes = set(sigungu_map.keys())
    invalid_codes = set(excluded_codes) - valid_codes
    if invalid_codes:
        print(f"잘못된 지역 코드가 포함되어 있습니다: {', '.join(invalid_codes)}")
        return False
    return True

def main():
    print_region_codes()
    excluded_codes = get_excluded_codes()

    while not validate_excluded_codes(excluded_codes) or not confirm_excluded_codes(excluded_codes):
        excluded_codes = get_excluded_codes()
    
    excluded_codes = [code for code in excluded_codes if code in sigungu_map]  # 다시 필터링

    # 모든 시/도 코드와 제외된 시/도 코드의 집합을 생성
    all_codes = set(sigungu_map.keys())
    remaining_codes = all_codes - set(excluded_codes)

    if not remaining_codes:
        print("여행 가능한 구역이 없습니다.")
        return

    # 제외된 포인트를 저장할 리스트
    excluded_points = []
    selected_points = set()  # 선택된 포인트를 저장할 집합

    # 대한민국 영역 내에 포함되는지 확인하고, 속한 시/군/구를 반환하는 함수
    def generate_random_point_within_korea():
        within_korea = False
        point = None
        attempts = 0
        max_attempts = 100

        while not within_korea and attempts < max_attempts:
            attempts += 1

            # 랜덤 위도 및 경도 생성 (대한민국의 범위 내에서)
            latitude = round(random.uniform(33.091, 38.614), 3)
            longitude = round(random.uniform(124.511, 131.874), 3)

            # Point 객체 생성
            point = Point(longitude, latitude)

            # 지리 데이터 프레임의 각 폴리곤(시/군/구) 내에 있는지 확인
            for index, row in gdf.iterrows():
                polygon = row['geometry']
                if point.within(polygon):  # Point가 Polygon 내부에 있는지 확인
                    sig_cd = row['SIG_CD']
                    region = row['SIG_KOR_NM']
                    
                    # sig_cd에 따라 시/도를 추가
                    sigungu = sigungu_map.get(sig_cd[:2], "")
                    if sigungu and sig_cd[:2] in excluded_codes:
                        excluded_points.append(f"{sigungu} {region}")
                    elif sigungu and sig_cd[:2] not in excluded_codes:
                        full_region = f"{sigungu} {region}" if sigungu else region
                        return point, sig_cd, full_region  # 점과 SIG_CD 반환

        return None, None, None  # 최대 시도 횟수를 넘긴 경우

    # 랜덤한 지역을 20개까지 찾기 위해 반복
    while len(selected_points) < 20:
        point, sig_cd, full_region = generate_random_point_within_korea()

        if point and full_region:
            selected_points.add(full_region)

    # 제외된 지역 확인
    excluded_provinces = [sigungu_map.get(code, "") for code in excluded_codes]
    excluded_provinces = [name for name in excluded_provinces if name]  # 빈 값 제거

    if selected_points:
        print("당첨 지역:")
        for region in selected_points:
            print(f"{region}")
        # print("제외된 지역:", ", ".join(excluded_provinces))
        # print("제외된 내역:")
        # for region in excluded_points:
        #     print(f"{region}")

if __name__ == "__main__":
    main()


다음 지역의 코드를 제외할 수 있습니다:
11: 서울특별시
26: 부산광역시
27: 대구광역시
28: 인천광역시
29: 광주광역시
30: 대전광역시
31: 울산광역시
41: 경기도
43: 충청북도
44: 충청남도
45: 전라북도
46: 전라남도
47: 경상북도
48: 경상남도
50: 제주도
51: 강원도
제외한 지역이 다음과 같습니까?
부산광역시 
대구광역시 
인천광역시 
광주광역시 
대전광역시 
울산광역시 
경기도 
충청북도 
충청남도 
전라북도 
전라남도 
제주도 
강원도 
당첨 지역:
경상북도 김천시
경상북도 영주시
경상남도 고성군
경상북도 안동시
경상남도 사천시
경상북도 의성군
경상북도 상주시
경상북도 영덕군
경상북도 경주시
경상북도 예천군
경상북도 청도군
경상남도 함안군
경상남도 거창군
경상남도 양산시
경상남도 함양군
경상북도 봉화군
경상남도 산청군
경상남도 하동군
경상남도 합천군
경상북도 영양군
